# 의사결정나무

## 17.1 의사결정나무란?
의사결정나무는 다양한 **의사결정 경로(decision path)** 와 **결과(outcome)** 를 나타내는 데 나무 구조를 사용

e.g) 스무고개
- 다리가 다섯 개 이상인가?
    - 예: 침대 밑에 숨어 있나?
        - 예: 소설 <살롯의 거미줄>에 등장하나?
            - 예: 거미
            
            - 아니요: 진드기
        
        - 아니요: 꿀을 만드나?
            - 예: 꿀벌
            
            - 아니요: 모기
    
    - 아니요: 맛있나?
        - 예: 소설 <살롯의 거미줄>에 등장하나?
            - 예: 돼지
            
            - 아니요: 소
            
        - 아니요: 호주의 5센트 동전의 뒷면에 등장하나?
            - 예: 바늘두더지
            
            - 아니요: 고양이
            
            
의사결정나무는 장점이 아주 많다. 이해하고 해석하기 쉽고, 예측할 때 사용하는 프로세스가 꽤 명백하다. 다른 모델과 다르게 숫자형 데이터(다리의 수)와 범주형 데이터(맛있다/맛없다)를 동시에 다룰 수 있고, 특정 변수의 값이 누락되어도 사용할 수 있다.

하지만 학습 데이터에 관해 **'최적의' 의사결정나무를 찾는 것은 계산적으로 무척 어려운 문제** 이다. (여기서는 최적의 의사결정나무를 찾는 대신 '어느 정도 좋은' 의사결정나무를 찾는 것으로 이 문제를 우회할 것, 방대한 데이터를 다룰 경우에는 이마저도 어려울 수 있다.)

의사결정나무는 **새로운 데이터에 대한 일반화 성능이 좋지 않게 오버피팅** 되기 쉽다.

많은 사람들은 의사결정나무를 **범주형 결과를 반환하는 분류나무(classification tree)** 와 **숫자형 결과를 반환하는 회귀나무(regression tree)**  로 나눈다. 

여기서는 분류나무를 중점적으로 살펴보고, 클래스 레이블이 있는 데이터로 ID3 알고리즘이 어떻게 의사결정나무를 구축하는지 살펴볼 것이다. 문제를 간단하게 하기 위해서 '웹사이트에 광고 A와 광고 B 중 어떤 것을 내보내야 할까?' 같이 결과값이 이진(binary)인 경우만 고려하자.

## 17.2 엔트로피
의사결정나무를 만들기 위해서선 어떤 질문을 물을 것이고 어떤 순서로 질문을 던질 것인지 정해야 한다. 나무의 각 단계에서 데이터에 대한 가능성이 완전히 배제되는 경우가 있고 아닌 경우가 있다. 

e.g. 동물의 다리가 5개 이하, 메뚜기일 가능성은 없고, 오리일 가능성은 있다.

**각 질문에 대한 답이 무엇이냐에 따라 가능성은 세분화된다.**

이상적으로, 예측하려는 대상에 대해 **가장 많은 정보를 담고 있는 질문을 고른 것이 좋다.** 질문에 대한 답이 '참'인 경우 결괏값이 항상 True이고 '거짓'인 경우 결괏값이 False인 질문이 딱 하나 있다면 그 질문을 먼저 물어봐야 한다. 질문을 던져 결괏값에 대한 새로운 정보를 전혀 주지 못한다면 좋은 질문이 아니다.

'얼마만큼의 정보를 담고 있는가'를 **엔트로피(entropy)** 라고 한다. 엔트로피가 **'무질서도(disorder)'** 를 의미한다. 이는 데이터의 **'불확실성(uncertainty)'** 을 나타낼 때도 같은 표현을 사용한다.

e.g.

데이터셋 S가 있고 각 데이터 포인트는 $C_1, ..., C_n$ 등 유한 개의 클래스 중 하나에 속한다고 해보자. 모든 데이터 포인트가 하나의 클래스에 속한다면 불확실성은 전혀 없고, 엔트로피는 낮다고 할 수 있다. 반면 모든 데이터 포인트가 모든 클래스에 고르게 분포되어 있다면 불확실성도 엔트로피도 높다.

위 개념을 수학적으로 표현하면 한 데이터 포인트가 클래스 $c_i$에 속할 확률을 $p_i$라고 하고 $0log0 = 0$이면 엔트로피는 다음과 같이 표기 할 수 있다.

$H(S) = -p_1 log_2 p_1 - ... -p_n log_2 p_n$

각 항 $-p_i log_2 p_i$의 값은 0보다 크거나 같으며, $p_i$의 값이 0 또는 1에 가까울때 0에 가까워 진다.

모든 $p_i$가 0 또는 1에 가까우면(대부분의 데이터 포인트가 하나의 클래스에 속하면) 엔트로피는 아주 작을 것이고, 대부분의 $p_i$가 0에 가깝지 않으면(데이터 포인트가 여러 클래스에 고르게 분포되어 있다면) 엔트로피는 큰 값을 가진다.

In [4]:
from typing import List
import math

def entropy(class_probabilities: List[float]) -> float:
    """클래스에 속할 확률을 입력하면 엔트로피를 계산하라."""
    return sum(-p * math.log(p, 2)
              for p in class_probabilities
              if p > 0)  # 확률이 0인 경우는 제외한다.

In [5]:
assert entropy([1.0]) == 0
assert entropy([0.5 , 0.5]) == 1
assert 0.81 < entropy([0.25, 0.75]) < 0.82

입력 데이터는 (input, label) 쌍으로 구성, 각 클래스 레이블의 확률은 별도 계산 필요. 엔트로피를 구할 때는 어떤 레이블에 어떤 확률값이 주어졌는지까지는 알 필요가 없고, 레이블과 무관하게 확률값만 알면 된다.

In [6]:
from typing import Any
from collections import Counter

def class_probabilities(labels: List[Any]) -> List[float]:
    total_count = len(labels)
    return [count / total_count
           for count in Counter(labels).values()]

def data_entropy(labels: List[Any]) -> float:
    return entropy(class_probabilities(labels))

In [7]:
assert data_entropy(['a']) == 0
assert data_entropy([True, False])  == 1
assert data_entropy([3, 4, 4, 4]) == entropy([0.25, 0.75])

## 17.3 파티션의 엔트로피
지금까지는 전체 데이터셋에 대한 엔트로피( == '불확실성')를 계산했다. 하지만 **의사결정나무의 각 단계는 데이터를 여러 개의 파티션(partition)으로 분할** 한다.

e.g. 
'다리가 5개 이상인가?' 질문은 다리가 5개 이상인 동물(e.g. 거미)에 대한 파티션과 그렇지 않은 동물(e.g. 바늘 두더지)에 대한 파티션으로 데이터를 나눈다.

하나의 데이터셋을 여러 개의 파티션으로 나누더라도 데이터셋 전체에 관한 엔트로피를 계산할 수 있는 방법이 필요해졌다. 이때 파티션 하나하나가 낮은 엔트로피를 가지는 경우에는 전반적인 엔트로피도 낮고, 파티션 하나하나가 높은 경우 전반적으로 엔트로피도 높아야 한다. 

수학적으로 표현하면 데이터 $S$를 $q_1, ..., q_m$의 비율을 가지는 파티션 $S_1, ..., S_m$로 나누는 경우 엔트로피는 **가중합** 으로 계산 가능

$H = q_1H(S_1) + ... + q_mH(S_m)$

이것을 코드로 작성하면 아래와 같다.

In [14]:
def partition_entropy(subsets: List[List[Any]]) -> float:
    """주어진 데이터의 파티션 안의 엔트로피를 반환"""
    total_count = sum(len(subset) for subset in subsets)
    
    return sum(data_entropy(subset) * len(subset) / total_count for subset in subsets)

**주의**
이 접근법의 문제는, 다양한 값을 가지는 변수를 사용해서 파티션을 나누는 경우 오버피팅이 되어 엔트로피가 낮아진다는 것이다. 

e.g. 
기존 데이터를 이용 은행 고객 중 주택담보대출을 갚지 못할 고객을 예측하는 의사결정나무를 만들려고 한다. 데이터에 고객들의 주민번호가 담겨 있는 변수가 있는데, 이 변수로 파티션을 나누면 파티션당 고객 한 명씩만 속해 엔트로피가 모두 0이 된다. (주민번호는 고유한 값이므로 한 클래스로 치면 클래스 한개 당 한명의 고객이 포함되게 되므로) 이렇게 파티션을 나누면 학습 데이터 외의 데이터를 잘 처리할 수 없다. 

따라서 의사결정나무를 사용할 때는 다양한 값을 가지는 변수를 최대한 피하거나, 변수의 여러 값을 적은 수의 버킷(bucket)으로 나눠서 선택 가능한 값의 종류를 줄이는 것이 좋다.

## 17.4 의사결정나무 만들기

In [17]:
from typing import NamedTuple, Optional

class Candidate(NamedTuple):
    level: str
    lang: str
    tweets: bool
    phd: bool
    did_well: Optional[bool] = None  # 레이블이 없는 데이터도 허용한다.

In [18]:
                      #  level     lang     tweets  phd  did_well
inputs = [Candidate('Senior', 'Java',   False, False, False),
          Candidate('Senior', 'Java',   False, True,  False),
          Candidate('Mid',    'Python', False, False, True),
          Candidate('Junior', 'Python', False, False, True),
          Candidate('Junior', 'R',      True,  False, True),
          Candidate('Junior', 'R',      True,  True,  False),
          Candidate('Mid',    'R',      True,  True,  True),
          Candidate('Senior', 'Python', False, False, False),
          Candidate('Senior', 'R',      True,  False, True),
          Candidate('Junior', 'Python', True,  False, True),
          Candidate('Senior', 'Python', True,  True,  True),
          Candidate('Mid',    'Python', False, True,  True),
          Candidate('Mid',    'Java',   True,  False, True),
          Candidate('Junior', 'Python', False, True,  False)
         ]

의사결정나무는 **결정 노드(decision node)** 와 **잎 노드(leaf node)** 구성 된다.

결정 노드는 질문을 주고 답변에 따라 다른 경로로 안내해 주고, 잎 노드는 예측값이 무엇인지 알려준다.

나무는 지금부터 기술할 ID3이라는 비교적 단순한 알고리즘에 기반해서 구축할 수 있다. 레이블이 있는 데이터와 파티션을 나눌 수 있는 변수의 목록이 주어졌다고 해보자.

- 모든 데이터 포인트의 클래스 레이블이 동일하다면 해당 레이블로 잎 노드를 만들고 종료하라.

- 변수 목록이 비어 있다면(즉 더 이상 물을 수 있는 질문이 없다면) 가장 빈도 수가 높은 클래스 레이블로 예측하는 잎 노드를 만들고 종료하라.

- 그게 아니면 각 변수로 데이터의 파티션을 나눠 보라.

- 파티션을 나눴을 때 엔트로피가 가장 낮은 변수를 택하라.

- 선택된 변수로 결정 노드를 생성하라.

위와 같은 방법을 'greedy(탐욕적)' 알고리즘이라 부르는데, 매 순간순간 가장 최적이라고 생각되는 선택을 하기 때문이다. 하지만 순간 중에는 최적의 선택이 아니더라도 나무 전체를 봤을 때는 더 좋은 선택인 경우가 있다. ID3 알고리즘은 그런 선택을 하지는 못한다. 하지만 상대적으로 이해가 쉽고, 구현도 쉬워 의사결정나무에 입문하기 위한 좋은 길잡이가 된다. 

위 inputs에 대한 결정 노드를 만들어보자. 총 네 가지 변수로 나무의 가지를 나눌 수 있다. 1단계는 가장 낮은 엔트로피를 반환하는 파티션을 찾는 것이다. 먼저 파티션을 나누는 함수를 작성해보자.

In [20]:
from typing import Dict, TypeVar 
from collections import defaultdict

T = TypeVar('T')  # 입력 데이터 inputs를 위한 타입

def partition_by(inputs: List[T], attribute: str) -> Dict[Any, List[T]]:
    """attribute에 따라 inputs의 파티션을 나누자."""
    partitions: Dict[Any, List[T]] = defaultdict(list)
    for input in inputs:
        key = getattr(input, attribute)  # 특정 attribute의 값을 불러오고
        partitions[key].append(input)  # 이 input에 맞는 파티션 추가
    return partitions

엔트로피 계산하는 코드

In [21]:
def partition_entropy_by(inputs: List[Any],
                        attribute: str,
                        label_attribute: str) -> float:
    """주어진 파티션에 대응되는 엔트로피를 계산"""
    # partitions는 inputs로 구성되어 있다.
    partitions = partition_by(inputs, attribute)
    
    # 하지만 partition_entropy에는 클래스 레이블만 있으면 된다.
    labels = [[getattr(input, label_attribute) for input in partition]
             for partition in partitions.values()]
    
    return partition_entropy(labels)

전체 데이터셋에 관해 엔트로피를 최소하하는 파티션을 찾으면 된다.

In [22]:
for key in ['level', 'lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(inputs, key, 'did_well'))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


In [24]:
assert 0.69 < partition_entropy_by(inputs, 'level', 'did_well')  < 0.70
assert 0.86 < partition_entropy_by(inputs, 'lang', 'did_well')   < 0.87
assert 0.78 < partition_entropy_by(inputs, 'tweets', 'did_well') < 0.79
assert 0.89 < partition_entropy_by(inputs, 'phd', 'did_well')    < 0.90

직급(level)에 관한 파티션을 나눌 때 엔트로피가 최솟값을 가진다. 직급의 가능한 값마다 가지를 나눠서 하위나무(subtree)를 만들자. 

직급이 Mid일 때 모든 후보가 True 클래스에 속하므로, Mid 하위 나무는 예측값으로 True를 반환하면 된다. 

한편 직급이 Senior일 때는 True와 False 클래스가 섞여 있기 때문에 다음과 같이 파티션을 나누는 과정을 반복해야 한다.

In [25]:
senior_inputs = [input for input in inputs if input.level == 'Senior']

In [28]:
assert 0.4 == partition_entropy_by(senior_inputs, 'lang', 'did_well')
assert 0.0 == partition_entropy_by(senior_inputs, 'tweets', 'did_well')
assert 0.95 < partition_entropy_by(senior_inputs, 'phd', 'did_well') < 0.96

앞의 결과에 따르면 tweets의 엔트로피 값은 0이므로 이에 따라 파티션을 나누자. 

이 경우 Senior 직급인 후보들이 트위터를 하면 항상 True이고, 트위터를 하지 않으면 항상 False가 된다.

마지막으로 Junior 직급 후보자들에 대해서도 같은 작업을 반복해 보면 phd를 기준으로 파티션을 나누게 되는데, phd의 경우 항상 False를 반환한다는 사실을 발견할 수 있다.

직급(level)?

- Senior:
    - Tweets?
        - Yes: 합격
        
        - No: 불합격
        
- Md: 합격

- Junior:
    - phd?
        - Yes: 불합격
        
        - No: 합격

## 17.5 종합하기
가장 먼저 해야 할 일은 나무를 어떻게 표현할지 결정하는 것이다. 여기서는 최대한 가벼운 표현법을 택해서 나무를 다음과 같이 정의하자.

- 한 개의 값을 예측하는 잎
- 특정 변수의 값에 따라 나뉜 하위나무와 처음 보는 변수의 값을 위해 사용할 기본값을 포함하는 분기

In [29]:
from typing import NamedTuple, Union, Any

class Leaf(NamedTuple):
    value: Any
        
class Split(NamedTuple):
    attribute: str
    subtrees: dict
    default_value: Any = None

DecisionTree = Union[Leaf, Split]

In [30]:
hiring_tree = Split('level', {   # 일단 직급('level')을 고려한다.
    'Junior': Split('phd', {     # 직급이 'Junior'이면 'phd'를 보자
        False: Leaf(True),       # 'phd'가 False이면 True라고 예측
        True: Leaf(False)        # 'phd'가 True이면 False라고 예측
    }),
    'Mid': Leaf(True),           # 직급이 'Mid'이면 True를 예측
    'Senior': Split('tweets', {  # 직급이 'Senior'이면 'tweets'를 보자
        False: Leaf(False),      # 'tweets'가 False이면 False라고 예측
        True: Leaf(True)         # 'tweets'가 True이면 True라고 예측
    })
})

한편, 새로운 후보자가 왔을 때 변수값 중 하나가 기존에 관찰되지 않은 것이면 어떻게 해야 할까? 예를 들어 새로운 후보자의 직급이 인턴(intern)인 경우에는 어떻게 해야 할까? 이런 경우 최대 빈도 레이블이 달린 default_value의 변수값을 이용할 것이다.

In [39]:
def classify(tree: DecisionTree, input: Any) -> Any:
    """의사결정나무로 새로운 입력값을 분류"""
    
    # 잎 노드이면 값을 반환해라
    if isinstance(tree, Leaf):
        return tree.value
    
    # 그게 아니라면 변수로 나무의 파티션을 나누자.
    # 키로 변수값, 값으로 하위나무를 나타내는 dict를 사용하면 된다.
    subtree_key = getattr(input, tree.attribute)
    
    if subtree_key not in tree.subtrees:  # 키에 해당하는 하위나무가 없다면
        return tree.default_value         # 기본값을 반환하라.
    
    subtree = tree.subtrees[subtree_key]  # 적합한 하위나무를 고르고
    
    return classify(subtree, input)       # 입력된 데이터를 분류한다.

학습용 데이터로부터 실제 나무를 구축

In [40]:
def build_tree_id3(inputs: List[Any],
                  split_attributes: List[str],
                  target_attribute: str) -> DecisionTree:
    # 레이블의 수를 센다.
    label_counts = Counter(getattr(input, target_attribute)
                          for input in inputs)
    most_common_label = label_counts.most_common(1)[0][0]
    
    # 레이블의 종류가 하나 뿐이라면 그것으로 예측하라.
    if len(label_counts) == 1:
        return Leaf(most_common_label)
    
    # 분기할 수 있는 변수가 더 이상 없다면 최대 빈도 레이블을 반환하라.
    if not split_attributes:
        return Leaf(most_common_label)
    
    # 그게 아니면 특정 변수로 분기하라.
    
    def split_entropy(attribute: str) -> float:
        """기준이 될 변수를 선책하는 함수"""
        return partition_entropy_by(inputs, attribute, target_attribute)
    
    best_attribute = min(split_attributes, key=split_entropy)
    
    partitions = partition_by(inputs, best_attribute)
    new_attributes = [a for a in split_attributes if a != best_attribute]
    
    # 재귀적으로 하위나무를 구축
    subtrees = {attribute_value : build_tree_id3(subset,
                                                 new_attributes,
                                                 target_attribute)
                for attribute_value, subset in partitions.items()}

    return Split(best_attribute, subtrees, default_value=most_common_label)

구축한 나무의 모든 잎 노드는 True or False로만 구성됨. 즉, 학습 데이터에 관해서만큼은 나무의 예측 오류가 0이라는 것인데, 한번도 관찰한 적 ㅇ벗는 새로운 테스트 데이터를 분류시킬 수 있고

In [41]:
tree = build_tree_id3(inputs,
                     ['level', 'lang', 'tweets', 'phd'],
                     'did_well')
# True로 예측해야 한다.
assert classify(tree, Candidate("Junior", "Java", True, False))

# False로 예측해야 한다.
assert not classify(tree, Candidate("Junior", "Java", True, True))

관찰된 적 없는 값이 변수에 등장하거나 변수 값 자체가 누락되었더라도 분류가 가능하다.

In [42]:
# True로 예측해야 한다.
assert classify(tree, Candidate("Inter", "Java", True, True))

여기서는 나무를 구축하는 방법을 보여 주는 것이 목적이었기 때문에 나무를 구축하는 데 데이터 전체를 사용했다. 하지만 우리가 실제로 어떤 작업을 하기 위해 모델을 구축하는 것이 목표였다면, 더 많은 데이터를 수입한 후 **데이터를 학습, 검증, 평가 데이터로 나눴을 것이다.**

## 17.6 랜덤 포레스트
의사결정나무를 쉽게 학습 데이터에 맞출 수 있는 것을 보면, 오버피팅되는 경향이 있는게 그리 놀라운 일이 아니다. 

오버피팅을 방지할 수 있는 대표적인 방법 중 하나로, 여러 개의 의사결정나무를 만들어 다수결로 결과물을 종합하는 **랜덤 포레스트(random forest)** 라는 것이 있다. 분류나무라면 나무의 결과로 과반 투표를 하게 되고, 회귀나무라면 예측값의 평균을 내면 된다.

지금까지는 구축된 나무에 랜덤성이 전혀 없었다. 어떻게 하면 랜덤하게 나무를 얻을 수 있을까?

15.6절에서 배운 '부트스트랩'에서 배운 대로 데이터를 부트스트랩하는 것이다. 

즉, inputs 전체를 이용해서 나무를 학습하는 것이 아니라 bootstrap_sample(inputs)의 결과물을 각 나무의 입력값으로 넣어 학습하는 것이다. 그러면 각 나무가 서로 다른 데이터로 구축되기 때문에 랜덤성이 생기게 된다. (이때 부가적인 이점은 샘플링되지 않은 데이터를 데스트 데이터로 이용할 수 있다는 점이다. 즉, 성능을 측정하는 방법만 잘 설계한다면 데이터 전체를 학습에 사용해도 된다는 것을 의미한다.) 이 방법을 **통합 부트스트랩(bootstrap aggregating)** 또는 **배깅(bagging)** 이라고 부른다.

두 번째 방법은 파티션을 나누는 변수 best_attribute에 랜덤성을 부여하는 것이다.

남아 있는 모든 변수 중에서 최적의 변수를 선택하는 것이 아니라 변수 중 일부만 선택하고 그 일부 중에서 최적의 변수를 선택하는 것이다.

In [43]:
# 만약 파티션 기준으로 사용할 변수가 얼마 남지 않았다면 전부 사용하자.
if len(split_candidates) <= self.num_split_candidates:
    sampled_split_candidates = split_candidates
    
# 아니라면 랜덤하게 변수를 선택
else:
    sampled_split_candidates = random.sample(split_candidates,
                                            self.num_split_candidates)
    
# 선택된 변수 중에서 가장 적절한 변수를 선택
best_attribute = min(sampled_split_candidates, key=split_entropy)

pratitions = partition_by(inputs, best_attribute)

NameError: name 'split_candidates' is not defined

이런 방법은 조금 더 광범위하게는 '앙상블 학습(ensemble learning)'의 일종인데, 성능이 떨어지는 (그리고 대부분, 편향은 높고 분산은 낮은) 여러 모델을 동시에 활용해서 전체적으로 성능이 좋은 모델을 구축한다.

## 17.7 더 공부해 보고 싶다면
- scikit-learn에는 많은 의사결정나무 모델이 구현되어 있다. 게다가 Random.ForestClassifier 등 다양한 앙상블 모듈이 있다. 

- XGBoost는 캐글과 같은 기계학습 대회에서 종종 우승하는 그래디언트 부스팅(gradient boosting) 나무를 학습한다.

- 우리는 이제 겨우 의사결정나무를 겉핥기했을 뿐이다. 위키피디아를 시작으로 더 공부해보자.